In [ ]:
import folium
import json

import pandas as pd
import numpy as np

# 지도 데이터
- 2019년도 데이터 : 서울만 뽑기

In [ ]:
geo_path2 = 'HangJeongDong_ver20191231.geojson'
geo_str2 = json.load(open(geo_path2, encoding='utf-8'))
geo_str2

In [ ]:
features_list = []

for dong in geo_str2["features"]:
    if dong["properties"]["adm_nm"].split()[0] == "서울특별시":
        dong["properties"]["adm_nm"] = dong["properties"]["adm_nm"].replace("·", ".")
        print(dong["properties"]["adm_nm"])
        features_list.append(dong)

In [ ]:
geo_str2["features"] = features_list

In [ ]:
geo_str2

# 분석용 데이터

In [ ]:
senior_house_data = pd.read_csv("1stClusteringData_1(cp949).csv", encoding="cp949")
senior_house_data = senior_house_data.fillna(0)
senior_house_data["행정동"] = senior_house_data["행정동"].str.replace("·", ".")
senior_house_data

In [ ]:
senior_house_data["매매"] = senior_house_data[["연립다세대_매매", "아파트_매매", "단독다가구_매매"]].mean(axis =1)
senior_house_data["전월세"] = senior_house_data[["연립다세대_전월세", "아파트_전월세", "단독다가구_전월세"]].mean(axis =1)
senior_house_data.head()

In [ ]:
senior_house_data = senior_house_data.drop(["연립다세대_매매", "연립다세대_전월세", "아파트_매매", 
                  "아파트_전월세", "단독다가구_매매", "단독다가구_전월세"], axis = 1)
# data = data.set_index(["구", "행정동"])
senior_house_data.head()

In [ ]:
senior_house_data["구동주소"] = "서울특별시 " + senior_house_data["구"] + " " + senior_house_data["행정동"]
senior_house_data

### 독거노인 분포

In [ ]:
map_senior_pop_heat = folium.Map(location=[37.5502, 126.982], zoom_start=10)
map_senior_pop_heat.choropleth(geo_data = geo_str2, 
               data=senior_house_data,
               columns=['구동주소',"독거노인"],
               key_on='properties.adm_nm',
               fill_color='PuRd')
map_senior_pop_heat

### 매매 분포

In [ ]:
map_sales_heat = folium.Map(location=[37.5502, 126.982], zoom_start=10)
map_sales_heat.choropleth(geo_data = geo_str2, 
               data=senior_house_data,
               columns=['구동주소',"매매"],
               key_on='properties.adm_nm',
               fill_color='PuRd')
map_sales_heat

### 전월세 분포

In [ ]:
map_rent_heat = folium.Map(location=[37.5502, 126.982], zoom_start=10)
map_rent_heat.choropleth(geo_data = geo_str2, 
               data=senior_house_data,
               columns=['구동주소',"전월세"],
               key_on='properties.adm_nm',
               fill_color='PuRd')
map_rent_heat

### 총 인구 분포

In [ ]:
population_data = pd.read_csv('전체인구_2019.txt', sep = "\t", engine='python', encoding = "utf-8", thousands = ',', header=[0,1,2])
population_data = population_data.iloc[:, [0,1,2,7]]
population_data.columns = ['기간', '자치구', '행정동', '계']
population_data = population_data.drop(population_data[(population_data['행정동']=='소계') | (population_data['행정동']=='합계')].index)
population_data = population_data.reset_index(drop=True)
population_data

In [ ]:
population_data['행정동'] = population_data['행정동'].str.replace("·", ".")

In [ ]:
population_data["구동주소"] = "서울특별시 " + population_data["자치구"] + " " + population_data["행정동"]

In [ ]:
population_map = folium.Map(location=[37.5502, 126.982], zoom_start=10)
population_map.choropleth(geo_data = geo_str2, 
               data=population_data,
               columns=['구동주소', '계'],
               key_on='properties.adm_nm',
               fill_color='PuRd')
population_map

### 1차 클러스터링 분포

In [ ]:
cluster_grade = pd.read_csv('clustering_basic(cp949).csv', encoding = "CP949")
cluster_grade[['구', '행정동', 'gmm_cluster_minmax']]

In [ ]:
cluster_grade['행정동'] = cluster_grade['행정동'].str.replace("·", ".")
cluster_grade["구동주소"] = "서울특별시 " + cluster_grade["구"] + " " + cluster_grade["행정동"]

In [ ]:
grade_map = folium.Map(location=[37.5502, 126.982], zoom_start=10)
grade_map.choropleth(geo_data = geo_str2, 
               data=cluster_grade,
               columns=['구동주소', 'gmm_cluster_minmax'],
               key_on='properties.adm_nm',
               fill_color='PuRd')
grade_map

### 총 노인인구 분포

In [ ]:
df_pop = pd.read_csv('olds_population(cp949).csv', encoding = "CP949")
df_pop

In [ ]:
df_pop_2019 = df_pop[df_pop['period']=='2019.4/4'].reset_index(drop=True)
df_pop_2019.drop(0, inplace=True)
df_pop_2019

In [ ]:
df_pop_2019['행정동'] = df_pop_2019['dong'].str.replace("·", ".")
df_pop_2019["구동주소"] = "서울특별시 " + df_pop_2019["gu"] + " " + df_pop_2019["행정동"]

In [ ]:
grade_map = folium.Map(location=[37.5502, 126.982], zoom_start=10)
grade_map.choropleth(geo_data = geo_str2, 
               data=df_pop_2019,
               columns=['구동주소', 'olds'],
               key_on='properties.adm_nm',
               fill_color='PuRd')
grade_map

In [ ]:
df_pop_2019['행정동'].unique()